In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Dec  5 17:10:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
#' ' means CPU whereas '/device:G:0' means GPU
# import tensorflow as tf
# tf.test.gpu_device_name()

In [ ]:
# tf.__version__

In [ ]:
# import time

In [ ]:
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

In [ ]:
# !kill -9 -1

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.layers import BatchNormalization
import keras
from keras import backend as K
import matplotlib.pyplot as plt
import sklearn

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import time
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.layers import BatchNormalization
import keras
from keras import backend as K
import matplotlib.pyplot as plt
import sklearn
import numpy as np
import PIL
import cv2
import os

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

In [7]:
classes = ['not fractured', 'fractured']

In [8]:
import os
from sklearn.model_selection import train_test_split
from skimage import io

In [9]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = io.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [10]:
# Replace these paths with the paths to your actual folders
fractured_folder = '/content/drive/My Drive/FracAtlas/images/fractured'
not_fractured_folder = '/content/drive/My Drive/FracAtlas/images/not_fractured'

In [11]:
# Load and preprocess fractured data
x_fractured = []
y_fractured = []

for img_filename in os.listdir(fractured_folder):
    img_path = os.path.join(fractured_folder, img_filename)
    img = load_img(img_path, target_size=(224, 224))  # Adjust target_size as needed
    img = img_to_array(img)
    img = img / 255.0
    x_fractured.append(img)
    y_fractured.append(1)  # Label 1 for fractured

In [12]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [13]:
# Load and preprocess not fractured data
x_not_fractured = []
y_not_fractured = []

for img_filename in os.listdir(not_fractured_folder):
    img_path = os.path.join(not_fractured_folder, img_filename)
    img = load_img(img_path, target_size=(224, 224))  # Adjust target_size as needed
    img = img_to_array(img)
    img = img / 255.0
    x_not_fractured.append(img)
    y_not_fractured.append(0)  # Label 0 for not fractured

In [14]:
X = x_fractured + x_not_fractured
y = y_fractured + y_not_fractured

In [15]:
X = np.array(X)
y = np.array(y)

In [16]:
X.shape

(4093, 224, 224, 3)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [18]:
num_samples, height, width, num_channels = x_train.shape
x_train_flattened = x_train.reshape(num_samples, -1)

In [19]:
from imblearn.over_sampling import RandomOverSampler

# Instantiate the RandomOverSampler
ros = RandomOverSampler(random_state=0)

# Resample the training data
x_train_resampled, y_train_resampled = ros.fit_resample(x_train_flattened, y_train)

In [20]:
# Reshape x_train_resampled back to its original shape
x_train_new = x_train_resampled.reshape(-1, height, width, num_channels)

# x_train_resampled_original now has the same shape as x_train


In [21]:
from collections import Counter

print("Class distribution after oversampling:", Counter(y_train_resampled))

Class distribution after oversampling: Counter({0: 2712, 1: 2712})


In [22]:
# Split the training data into train and validation
x_train, x_val, y_train, y_val = train_test_split(x_train_new, y_train_resampled, test_size=0.1, random_state=42)

In [23]:
# One-Hot Encode Labels
num_classes = len(classes)
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
# Assuming y_train_one_hot has shape (num_samples, num_classes)
# Now, find the column indices corresponding to 'fractured' and 'not fractured'
fractured_index = classes.index('fractured')
not_fractured_index = classes.index('not fractured')
print("Index for 'fractured':", fractured_index)
print("Index for 'not fractured':", not_fractured_index)


Index for 'fractured': 1
Index for 'not fractured': 0


In [ ]:
# Check the shapes of the datasets
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_val shape:", x_val.shape)
print("y_val shape:", y_val.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (4881, 224, 224, 3)
y_train shape: (4881, 2)
x_val shape: (543, 224, 224, 3)
y_val shape: (543, 2)
x_test shape: (819, 224, 224, 3)
y_test shape: (819, 2)


In [ ]:
# Assuming y_test is a NumPy array of shape (600, 2)
class_counts1 = np.sum(y_train, axis=0)
class_counts2 = np.sum(y_test, axis=0)
class_counts3 = np.sum(y_val, axis=0)

In [ ]:
class_counts1 #3946 not fractured, 4030 fractured

array([2425., 2456.], dtype=float32)

In [ ]:
class_counts2 #240 not fractured, 360 fractured.

array([664., 155.], dtype=float32)

In [ ]:
class_counts3

array([287., 256.], dtype=float32)

In [ ]:
y_test.dtype

dtype('float32')

In [24]:
from keras.models import load_model
from keras.layers import Lambda
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [25]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False,input_shape=(224,224,3))
# mark loaded layers as not trainable
for layer in model.layers:
	layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
# dense1 = Dense(1024, activation='relu')(flat1)
# dropout1 = Dropout(0.3)(dense1)  # Add dropout for regularization
# dense2 = Dense(512, activation='relu')(dropout1)
# dropout2 = Dropout(0.3)(dense2)  # Add dropout for regularization
# output = Dense(2, activation='softmax')(dropout2)

x=Dense(1024,activation='relu')(flat1) # FC layer 1
x=Dense(64,activation='relu')(x)   # FC layer 2
output = Dense(2, activation='softmax')(flat1)

model = Model(inputs=model.inputs, outputs=output)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)  # Adjust learning rate as needed
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Display the model summary
model.summary()

74836368/74836368 [==============================] - 3s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  

In [26]:
from tensorflow.keras.callbacks import EarlyStopping

In [27]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=200, callbacks=[early_stopping], verbose=1)
# model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=200, verbose=1)

Epoch 1/50
25/25 [==============================] - 36s 660ms/step - loss: 41.9219 - accuracy: 0.6462 - val_loss: 9.6290 - val_accuracy: 0.8214
Epoch 2/50
25/25 [==============================] - 7s 298ms/step - loss: 8.1113 - accuracy: 0.8220 - val_loss: 2.6088 - val_accuracy: 0.8895
Epoch 3/50
25/25 [==============================] - 7s 299ms/step - loss: 2.6323 - accuracy: 0.8984 - val_loss: 1.6143 - val_accuracy: 0.9319
Epoch 4/50
25/25 [==============================] - 7s 298ms/step - loss: 0.6764 - accuracy: 0.9551 - val_loss: 1.0294 - val_accuracy: 0.9448
Epoch 5/50
25/25 [==============================] - 7s 299ms/step - loss: 0.2006 - accuracy: 0.9820 - val_loss: 0.9467 - val_accuracy: 0.9595
Epoch 6/50
25/25 [==============================] - 7s 299ms/step - loss: 0.0906 - accuracy: 0.9910 - val_loss: 0.7712 - val_accuracy: 0.9576
Epoch 7/50
25/25 [==============================] - 7s 300ms/step - loss: 0.0410 - accuracy: 0.9936 - val_loss: 0.6017 - val_accuracy: 0.9687
Epoc

In [ ]:
# !pip install efficientnet

In [ ]:
# from efficientnet.tfkeras import EfficientNetB0

In [ ]:
# model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# # mark loaded layers as not trainable
# for layer in model.layers:
# 	layer.trainable = False
# # add new classifier layers
# flat1 = Flatten()(model.layers[-1].output)
# #x=Dense(1024,activation='relu')(flat1) # FC layer 1
# #x=Dense(64,activation='relu')(x)   # FC layer 2
# output = Dense(2, activation='softmax')(flat1)
# model = Model(inputs=model.inputs, outputs=output)

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, batch_size=200,verbose=1)


In [28]:
import sklearn
from sklearn.metrics import confusion_matrix

In [29]:
test_loss, test_acc = model.evaluate(np.array(x_test), np.array(y_test), verbose=0)
print(test_acc)
##Evaluating Sensitivity, Accuracy and Kappa scores
y_prob = model.predict(x_test)
Y_pred = y_prob.argmax(axis=-1)

0.8717948794364929
26/26 [==============================] - 5s 54ms/step


In [ ]:
y_test_flat = y_test.argmax(axis=-1)
y_test_flat.shape

(819,)

In [ ]:
cm1 = confusion_matrix(y_test_flat,Y_pred)
print("confusion matrix \n",cm1)

confusion matrix 
 [[641  23]
 [ 82  73]]


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
import pandas as pd

In [ ]:
print(pd.DataFrame(classification_report(y_test_flat,Y_pred,output_dict=True)).T)
Kappa=sklearn.metrics.cohen_kappa_score(y_test_flat,Y_pred)
print('Kappa=',Kappa)

              precision    recall  f1-score     support
0              0.886584  0.965361  0.924297  664.000000
1              0.760417  0.470968  0.581673  155.000000
accuracy       0.871795  0.871795  0.871795    0.871795
macro avg      0.823500  0.718165  0.752985  819.000000
weighted avg   0.862706  0.871795  0.859454  819.000000
Kappa= 0.5108612187089399


In [ ]:
model.save('/content/drive/My Drive/bonedata/DenseNet201_original_fracatlas.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Just conversion, no quantization

In [ ]:
from google.colab import files

In [ ]:
#convert tf.Keras to tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
mobilenet_noquant_model = converter.convert()

In [ ]:
#write the model to a tflite file as a binary file
mobilenet_noquant_tflite = "/content/drive/My Drive/bonedata" + "/DenseNet201_noquant_fracatlas.tflite"
with open(mobilenet_noquant_tflite, "wb") as f:
  f.write(mobilenet_noquant_model)
  # tflite_model1.save(mobilenet_noquant_file)
  # files.download(mobilenet_noquant_file)

# Convert to 16 FP

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_test).batch(1).take(100):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_types = [tf.float16]
# Set the input and output tensors to uint8 (APIs added in r2.3)
# converter.inference_input_type = tf.float32
# converter.inference_output_type = tf.float32
mobilenet_16fpquant_model = converter.convert()

In [ ]:
mobilenet_16fpquant_tflite = "/content/drive/My Drive/bonedata/" + "DenseNet201_float16quant_full_fracatlas.tflite"
with open(mobilenet_16fpquant_tflite, "wb") as f:
  f.write(mobilenet_16fpquant_model)

#Convert to INT8 FULL QUANT

In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(300):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8

mobilenet_int8quant_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
#write the model to a tflite file as binary file
mobilenet_int8quant_tflite = "/content/drive/My Drive/bonedata/" + "DenseNet201_int8quant_full_fracatlas.tflite"
with open(mobilenet_int8quant_tflite, "wb") as f:
  f.write(mobilenet_int8quant_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_content=mobilenet_int8quant_model)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.float32'>
output:  <class 'numpy.float32'>


# Run the TFLite Models

Now we'll run inferences using the TensorFlow Lite Interpreter to compare the model accuracies.

First, we need a function that runs inference with a given model and images, and then returns the predictions:

In [ ]:
# mobilenet_noquant_tflite = "/content/drive/My Drive/bonedata" + "/DenseNet201_noquant_fracatlas.tflite"

In [ ]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
  global x_test

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices), 2), dtype=np.float32)

  start_time = time.time()
  for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index] #has dimension of 2

    # Check if the input type is quantized, then rescale input data to uint8
    # if input_details['dtype'] == np.uint8:
    #   input_scale, input_zero_point = input_details["quantization"]
    #   test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output
  end_time = time.time()
  latency = (end_time - start_time) / len(test_image_indices)

  return [predictions, latency]

Evaluate the models on all images

In [ ]:
# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global x_test
  global y_test

  test_image_indices = range(x_test.shape[0])
  predictions, latency = run_tflite_model(tflite_file, test_image_indices)
  # print(predictions)
  true_class_indices = np.argmax(y_test, axis=-1)
  predicted_class_indices = np.argmax(predictions, axis=-1)

  accuracy = (np.sum(true_class_indices == predicted_class_indices) * 100) / len(x_test)

  print(pd.DataFrame(classification_report(true_class_indices,predicted_class_indices,output_dict=True)).T)
  Kappa=sklearn.metrics.cohen_kappa_score(true_class_indices,predicted_class_indices)
  print('Kappa=',Kappa)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (model_type, accuracy, len(x_test)))
  print('%s model Latency: %.2f ms per inference' % (model_type, latency*1000))
  cm1 = confusion_matrix(true_class_indices, predicted_class_indices)
  print("confusion matrix \n",cm1)

Evaluate the float32 model:

In [ ]:
evaluate_model(mobilenet_noquant_tflite, model_type="Float32")

              precision    recall  f1-score     support
0              0.886584  0.965361  0.924297  664.000000
1              0.760417  0.470968  0.581673  155.000000
accuracy       0.871795  0.871795  0.871795    0.871795
macro avg      0.823500  0.718165  0.752985  819.000000
weighted avg   0.862706  0.871795  0.859454  819.000000
Kappa= 0.5108612187089399
Float32 model accuracy is 87.1795% (Number of test samples=819)
Float32 model Latency: 179.56 ms per inference
confusion matrix 
 [[641  23]
 [ 82  73]]


Evaluate float16 model:

In [ ]:
evaluate_model(mobilenet_16fpquant_tflite, model_type="Float16")

              precision    recall  f1-score     support
0              0.886740  0.966867  0.925072  664.000000
1              0.768421  0.470968  0.584000  155.000000
accuracy       0.873016  0.873016  0.873016    0.873016
macro avg      0.827581  0.718918  0.754536  819.000000
weighted avg   0.864348  0.873016  0.860522  819.000000
Kappa= 0.5141129492298917
Float16 model accuracy is 87.3016% (Number of test samples=819)
Float16 model Latency: 176.67 ms per inference
confusion matrix 
 [[642  22]
 [ 82  73]]


Evaluate INT8 model:

In [ ]:
evaluate_model(mobilenet_int8quant_tflite, model_type="int8")

              precision    recall  f1-score     support
0              0.815756  0.966867  0.884907  664.000000
1              0.312500  0.064516  0.106952  155.000000
accuracy       0.796093  0.796093  0.796093    0.796093
macro avg      0.564128  0.515692  0.495929  819.000000
weighted avg   0.720512  0.796093  0.737675  819.000000
Kappa= 0.04510133837872554
int8 model accuracy is 79.6093% (Number of test samples=819)
int8 model Latency: 170.31 ms per inference
confusion matrix 
 [[642  22]
 [145  10]]


In [ ]:
# Size comparison
import os
int8size = os.path.getsize('/content/drive/My Drive/bonedata/DenseNet201_int8quant_full_fracatlas.tflite')
non_quantized_size = os.path.getsize('/content/drive/My Drive/bonedata/DenseNet201_noquant_fracatlas.tflite')
float16_size = os.path.getsize('/content/drive/My Drive/bonedata/DenseNet201_float16quant_full_fracatlas.tflite')

print(f'Non-Quantized Model Size: {non_quantized_size} bytes')
print(f'float16 Model Size: {float16_size} bytes')
print(f'int8 Model Size: {int8size} bytes')
